In [0]:
%run ./utilityFunctions

In [0]:
# Initialize variables
from functools import reduce
from pyspark.sql.functions import sum, avg, max, min, count, expr, when, col
dateTable = spark.table("gold.datetable")
cityTable = spark.table("gold.capitalcitylist")
currentFactTable = spark.table("silver.currentfacttable")
currentWeatherFactTable = spark.table("silver.currentweatherfacttable")
hourlyFactTable = spark.table("silver.hourlyfacttable")
hourlyWeatherFactTable = spark.table("silver.hourlyweatherfacttable")
dailyFactTable = spark.table("silver.dailyfacttable")
dailyTempFactTable = spark.table("silver.dailytempfacttable")
dailyFeelsLikeFactTable = spark.table("silver.dailyfeelslikefacttable")
dailyWeatherFactTable = spark.table("silver.dailyweatherfacttable")


def filterTableLastNDays(factTable, numDaysBack):
    dayList = getLastNDays(numDaysBack)
    joinDF = factTable.join(dateTable).where(dateTable["date_queried"] == factTable["time_data_queried_utc"]).where(reduce(lambda a, b: a|b, (dateTable['date'].like('%'+pattern+'%') for pattern in dayList)))
    return joinDF


def filterTableLastNHrs(factTable, numHrsBack):
    hrList = getLastNHrs(numHrsBack)
    joinDF = factTable.join(dateTable).where(dateTable["date_queried"] == factTable["time_data_queried_utc"]).where(reduce(lambda a, b: a|b, (dateTable['date'].like('%'+pattern+'%') for pattern in hrList)))
    return joinDF
  

def filterMaxMin(filteredFactTable, category):
    maxCategoryName = "max_" + category
    maxCategoryColumnName = "max_" + category + "_capital"
    minCategoryName = "min_" + category
    minCategoryColumnName = "min_" + category + "_capital"
    maxExpr = "max_by(capital, " + category + ") as " + maxCategoryColumnName
    minExpr = "min_by(capital, " + category + ") as " + minCategoryColumnName
    maxMinTable = filteredFactTable.groupBy("date").agg(max(category).alias(maxCategoryName), expr("max_by(capital, temp) as max_temp_capital"), min(category).alias(minCategoryName), expr("min_by(capital, temp) as min_temp_capital"))
    return maxMinTable


def filterWeatherTypeCount(filteredWeatherFactTable, weatherType):
    columnName = "count_" + weatherType
    weatherCountTable = filteredWeatherFactTable.groupBy("date").agg(count(when(filteredWeatherFactTable.description_short == weatherType, True)).alias(columnName))
    return weatherCountTable


In [0]:
currentFactTable7Days = filterTableLastNDays(currentFactTable, 7)
currentFactTable24Hrs = filterTableLastNHrs(currentFactTable, 24)
currentWeatherFactTable7Days = filterTableLastNDays(currentWeatherFactTable, 7)
currentWeatherFactTable24Hrs = filterTableLastNHrs(currentWeatherFactTable, 24)
hourlyFactTable7Days = filterTableLastNDays(hourlyFactTable, 7)
hourlyFactTable24Hrs = filterTableLastNHrs(hourlyFactTable, 24)
hourlyWeatherFactTable7Days = filterTableLastNDays(hourlyWeatherFactTable, 7)
hourlyWeatherFactTable24Hrs = filterTableLastNHrs(hourlyWeatherFactTable, 24)
dailyFactTable7Days = filterTableLastNDays(dailyFactTable, 7)
dailyFactTable24Hrs = filterTableLastNHrs(dailyFactTable, 24)
dailyTempFactTable7Days = filterTableLastNDays(dailyTempFactTable, 7)
dailyTempFactTable24Hrs = filterTableLastNHrs(dailyTempFactTable, 24)
dailyFeelsLikeFactTable7Days = filterTableLastNDays(dailyFeelsLikeFactTable, 7)
dailyFeelsLikeFactTable24Hrs = filterTableLastNHrs(dailyFeelsLikeFactTable, 24)
dailyWeatherFactTable7Days = filterTableLastNDays(dailyWeatherFactTable, 7)
dailyWeatherFactTable24Hrs = filterTableLastNHrs(dailyWeatherFactTable, 24)


In [0]:
currentMaximumMinimum24Hrs = filterMaxMin(currentFactTable24Hrs, "temp")
display(currentMaximumMinimum24Hrs)

In [0]:
currentMaximumMinimum7Days = filterMaxMin(currentFactTable7Days, "temp")
display(currentMaximumMinimum7Days)

In [0]:
weatherCountPast7Days = filterWeatherTypeCount(currentWeatherFactTable7Days, "Clouds")
display(weatherCountPast7Days)

In [0]:
weatherCountPast24Hrs = filterWeatherTypeCount(currentWeatherFactTable24Hrs, "Clear")
display(weatherCountPast24Hrs)